# Plots and figures Simulations

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import os.path as path
import os
import socket
import sys

In [ ]:
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
print("Your style sheets are located at: {}".format(path.join(mpl.__path__[0], 'mpl-data', 'stylelib')))

import numpy as np
import pandas as pd
# import pingouin
import ptitprince as pt
# import scipy.stats
import seaborn as sns

%matplotlib inline

In [ ]:
sys.path.append('..')

from configs.configs import get_config_dict
from helpers.figures import set_size

In [ ]:
all_styles = [
    'seaborn-dark',
    'seaborn-darkgrid',
    'seaborn-ticks', 'fivethirtyeight', 'seaborn-whitegrid', 'classic',
    '_classic_test', 'fast', 'seaborn-talk', 'seaborn-dark-palette', 'seaborn-bright',
    'seaborn-pastel', 'grayscale', 'seaborn-notebook', 'ggplot', 'seaborn-colorblind',
    'seaborn-muted', 'seaborn', 'Solarize_Light2', 'seaborn-paper',
    'bmh', 'tableau-colorblind10', 'seaborn-white', 'dark_background', 'seaborn-poster', 'seaborn-deep'
]

In [ ]:
plt.style.available

In [ ]:
sns.set_style('whitegrid')
# plt.style.use('tex')

In [ ]:
# However, the figsize argument takes inputs in inches and we have the width of our document in pts. To set the figure size we construct a function to convert from pts to inches and to determine an aesthetic figure height using the golden ratio:

### Synthetic covariance structures

In [ ]:
sim_cfg = get_config_dict(
    data_set_name='sim',  # not used here
    experiment_data='',  # not used here
    hostname=socket.gethostname()
)

In [ ]:
from benchmarks.simulations.plotters.plot_covariance_structures import _plot_synthetic_covariance_structures

In [ ]:
_plot_synthetic_covariance_structures(
    config_dict=sim_cfg,
    # figures_save_basedir=os.path.join(cfg['project-basedir'], 'opk20_hivemind_paper_1', 'figures', 'synthetic_data')
    figures_save_basedir="/Users/onnokampman/Developer/"
)

### TVFC estimates time series

In [ ]:
sim_cfg = get_config_dict(
    data_set_name='d2',
    experiment_data='N0400_T0200',
    hostname=socket.gethostname()
)

#### Fig. 4

In [ ]:
from benchmarks.simulations.plotters.plot_TVFC_estimates import _plot_d2_all_covariance_structures

In [ ]:
_plot_d2_all_covariance_structures(
    config_dict=sim_cfg,
    signal_to_noise_ratio=0.0,
    connectivity_metric='correlation',
    time_series_noise_type='no_noise',
    i_trial=0,
    data_split='all',
    # figures_savedir=os.path.join(
    #     cfg['figures-basedir'], noise_type, data_split, "TVFC_estimates", f'trial_{i_trial:03d}'
    # )
)

### Quantitative results

#### Fig. 5

In [ ]:
# data_set_name = 'd2'
# data_set_name = 'd3d'
# data_set_name = 'd3s'
# data_set_name = 'd4s'
# data_set_name = 'd6s'
# data_set_name = 'd9s'
data_set_name = 'd15s'

# experiment_data = 'N0400_T0200'
# experiment_data = 'N0200_T0200'
experiment_data = 'N0400_T0010'

# noise_type = 'no_noise'
noise_type = 'HCP_noise_snr_2'

n_trials = int(experiment_data[-4:])
data_split = 'all'

# perform_metric = 'correlation_RMSE'
perform_metric = 'correlation_matrix_RMSE'

sim_cfg = get_config_dict(
    data_set_name=data_set_name,
    experiment_data=experiment_data,
    hostname=socket.gethostname()
)

In [ ]:
from benchmarks.simulations.plotters.plot_quantitative_results import _plot_quantitative_results

In [ ]:
matplotlib.colormaps.items

In [ ]:
_plot_quantitative_results(
    config_dict=sim_cfg,
    performance_metric=perform_metric,
    noise_type=noise_type,
    data_split=data_split,
    figures_savedir='/Users/onnokampman/Developer/'
)

In [ ]:
cmap = matplotlib.colormaps['tab10']
# cmap = matplotlib.colormaps.get_cmap('tab10')

cmap
# cmap(0)

### Imputation study

In [ ]:
from benchmarks.simulations.imputation_study.plot_LEOO_likelihoods import _plot_raincloud

In [ ]:
data_set_name = 'd2'

In [ ]:
data_split = 'LEOO'
noise_type = 'no_noise'
sim_cfg = get_config_dict(
    data_set_name=data_set_name,
    experiment_data='N0400_T0200',
    hostname=socket.gethostname()
)

In [ ]:
# Plot all covariance structures for a single method.
model_name = 'DCC_joint'

likelihoods_filename = f'{data_split:s}_{noise_type:s}_likelihoods_{model_name:s}.csv'
test_likelihoods_savedir = os.path.join(sim_cfg['git-results-basedir'], 'imputation_study')
test_likelihoods_savepath = os.path.join(test_likelihoods_savedir, likelihoods_filename)

if os.path.exists(test_likelihoods_savepath):
    logging.info(f"Loading '{test_likelihoods_savepath:s}'...")
    likelihoods_df = pd.read_csv(test_likelihoods_savepath, index_col=0)  # (n_trials, n_covs_types)
    likelihoods_df = likelihoods_df.loc[:, sim_cfg['plot-covs-types']]

    # Update covs types labels for plots.
    likelihoods_df.columns = likelihoods_df.columns.str.replace('periodic_1', 'periodic (slow)')
    likelihoods_df.columns = likelihoods_df.columns.str.replace('periodic_3', 'periodic (fast)')
    likelihoods_df.columns = likelihoods_df.columns.str.replace('checkerboard', 'boxcar')
    likelihoods_df.columns = likelihoods_df.columns.str.replace('_', ' ')

    # print(likelihoods_df)
    _plot_raincloud(
        config_dict=sim_cfg,
        test_likelihoods_df=likelihoods_df,
        noise_type=noise_type,
        palette=sim_cfg['plot-covs-types-palette'],
        model_name=model_name
    )

In [ ]:
# Plot all methods for a single covariance structure.
covs_type = 'null'
# covs_type = 'periodic_1'
n_trials = 200

covs_type_df = pd.DataFrame(
    np.nan,
    index=np.arange(n_trials),
    columns=sim_cfg['plot-models']
)
for model_name in sim_cfg['plot-models']:
    likelihoods_filename = f'{data_split:s}_{noise_type:s}_likelihoods_{model_name:s}.csv'
    test_likelihoods_savedir = os.path.join(sim_cfg['git-results-basedir'], 'imputation_study')
    test_likelihoods_savepath = os.path.join(test_likelihoods_savedir, likelihoods_filename)
    if os.path.exists(test_likelihoods_savepath):
        likelihoods_df = pd.read_csv(test_likelihoods_savepath, index_col=0)  # (n_trials, n_train_covs_types)
        # assert likelihoods_df.shape == (n_trials, len(sim_cfg['train-covs-types']))
        likelihoods_df = likelihoods_df.loc[:, sim_cfg['plot-covs-types']]
        # print(likelihoods_df)
        covs_type_df.loc[:, model_name] = likelihoods_df.loc[:, covs_type]
    else:
        logging.warning(f"{test_likelihoods_savepath:s} not found.")
print(covs_type_df)

# Update model names labels for plots.
covs_type_df.columns = covs_type_df.columns.str.replace('SVWP_joint', 'WP')
covs_type_df.columns = covs_type_df.columns.str.replace('SVWP', 'WP')
covs_type_df.columns = covs_type_df.columns.str.replace('SW_cross_validated', 'SW-CV')
if data_set_name == 'd2':
    covs_type_df.columns = covs_type_df.columns.str.replace('_joint', '')
else:
    covs_type_df.columns = covs_type_df.columns.str.replace('_joint', '-J')
    covs_type_df.columns = covs_type_df.columns.str.replace('_bivariate_loop', '-BL')
covs_type_df.columns = covs_type_df.columns.str.replace('_', ' ')

_plot_raincloud(
    config_dict=sim_cfg,
    test_likelihoods_df=covs_type_df,
    noise_type=noise_type,
    palette=sim_cfg['plot-methods-palette'],
    covs_type=covs_type,
    figures_savedir="/Users/onnokampman/Developer/"
)

### Learned kernel lengthscales

In [ ]:
from benchmarks.simulations.plotters.plot_kernel_lengthscales import _plot_kernel_lengthscales

In [ ]:
data_split = 'all'
kernel_param = 'kernel_lengthscales'
model_name = 'SVWP'
noise_type = 'no_noise'

sim_cfg = get_config_dict(
    data_set_name='d2',
    experiment_data='N0400_T0200',
    hostname=socket.gethostname()
)

In [ ]:
kernel_lengthscales_df_filepath = os.path.join(
    sim_cfg['git-results-basedir'], noise_type, data_split, f'{model_name:s}_{kernel_param:s}_kernel_params.csv'
)
kernel_lengthscales_df = pd.read_csv(
    kernel_lengthscales_df_filepath,
    index_col=0
)
kernel_lengthscales_df = kernel_lengthscales_df.loc[:, sim_cfg['plot-covs-types']]

# Update covs types labels for plots.
kernel_lengthscales_df.columns = kernel_lengthscales_df.columns.str.replace('periodic_1', 'periodic (slow)')
kernel_lengthscales_df.columns = kernel_lengthscales_df.columns.str.replace('periodic_3', 'periodic (fast)')
kernel_lengthscales_df.columns = kernel_lengthscales_df.columns.str.replace('checkerboard', 'boxcar')
kernel_lengthscales_df.columns = kernel_lengthscales_df.columns.str.replace('_', ' ')

_plot_kernel_lengthscales(
    config_dict=sim_cfg,
    model_name=model_name,
    kernel_lengthscales_df=kernel_lengthscales_df,
    figure_savedir='/Users/onnokampman/Developer/'
)

### Optimal window lengths

In [ ]:
from benchmarks.simulations.plotters.plot_optimal_window_lengths import _plot_optimal_window_lengths

In [ ]:
optimal_window_lengths_df = pd.read_csv(
    os.path.join(
        sim_cfg['git-results-basedir'], noise_type, data_split, 'optimal_window_lengths.csv'
    ),
    index_col=0
)
optimal_window_lengths_df = optimal_window_lengths_df.loc[:, sim_cfg['plot-covs-types']]

# Update covs types labels for plots.
optimal_window_lengths_df.columns = optimal_window_lengths_df.columns.str.replace('periodic_1', 'periodic (slow)')
optimal_window_lengths_df.columns = optimal_window_lengths_df.columns.str.replace('periodic_3', 'periodic (fast)')
optimal_window_lengths_df.columns = optimal_window_lengths_df.columns.str.replace('checkerboard', 'boxcar')
optimal_window_lengths_df.columns = optimal_window_lengths_df.columns.str.replace('_', ' ')

_plot_optimal_window_lengths(
    config_dict=sim_cfg,
    optimal_window_lengths_df=optimal_window_lengths_df,
    figure_savedir='/Users/onnokampman/Developer/'
)
